In [ ]:
from celluloid import Camera
from IPython.display import HTML
import geopandas as gpd
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
import os
import pandas as pd

In [ ]:
df = pd.read_csv('5_Day_Mean_Incidence_Rate_Per_NY_Zip_Code.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by='Timestamp')
min_date = df['Timestamp'].min().date().strftime('%B %d')
max_date = df['Timestamp'].max().date().strftime('%B %d %Y')

In [ ]:
gdf = gpd.read_file('data/ZIP_CODE_040114/ZIP_CODE_040114.shp')
gdf = gdf.rename(columns={'ZIPCODE': 'Zip Code'})
gdf['Zip Code'] = gdf['Zip Code'].astype('int')

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.title('5 Day Mean Incidence Rate by NYC Zip Code\n' + min_date + ' to ' + max_date, fontsize=20)
plt.xticks([])
plt.yticks([])

camera = Camera(fig)
ax = fig.add_subplot(111)

for by, group in df.groupby(['Timestamp']):
    zip_map = gdf.merge(group, on='Zip Code')
    zip_map.plot(column='5 Day Mean Incidence Rate', ax=ax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('Longitude', fontsize=20)
    ax.set_ylabel('Latitude',fontsize=20)
    camera.snap()

animation = camera.animate(interval=2*200)

file_name = 'map-animation.mp4'
animation.save(file_name)

clip = VideoFileClip(file_name)
clip.write_gif("map-animation.gif")
os.remove(file_name)

HTML(animation.to_html5_video())